In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q simpletransformers scikit-learn pandas numpy tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from simpletransformers.classification import ClassificationModel
import torch
import logging
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical


In [4]:
use_cuda = torch.cuda.is_available()
print("GPU Available:", use_cuda)


GPU Available: True


In [5]:
train_df = pd.read_csv("/content/drive/MyDrive/meme_classifications/ocr_text/train/STask_D_train.csv")
eval_df = pd.read_csv("/content/drive/MyDrive/meme_classifications/test_ocr_text/STask-D(index,text)test.csv")
train_df.columns = ["index", "text", "labels"]
eval_df.columns = ["index", "text"]


In [6]:
label_encoder = LabelEncoder()
train_df["labels"] = label_encoder.fit_transform(train_df["labels"])
num_labels = len(label_encoder.classes_)


In [7]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df["text"], train_df["labels"], test_size=0.1, random_state=42
)
train_data = pd.DataFrame({"text": train_texts, "labels": train_labels})
val_data = pd.DataFrame({"text": val_texts, "labels": val_labels})


In [8]:
albert_model = ClassificationModel(
    "albert",
    "albert-base-v2",
    num_labels=num_labels,
    args={
        "reprocess_input_data": True,
        "overwrite_output_dir": True,
        "num_train_epochs": 4,
        "learning_rate": 1e-5,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "max_seq_length": 128,
        "save_model_every_epoch": False,
        "save_eval_checkpoints": False,
        "use_multiprocessing": False,
        "fp16": False,
        "evaluate_during_training": True,
        "use_early_stopping": True,
        "early_stopping_patience": 2,
        "output_dir": "outputs_subtaskD",
    },
    use_cuda=use_cuda,
)

albert_model.train_model(train_data, eval_df=val_data)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

Running Epoch 3 of 4:   0%|          | 0/228 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Running Epoch 4 of 4:   0%|          | 0/228 [00:00<?, ?it/s]

0it [00:00, ?it/s]

(912,
 defaultdict(list,
             {'global_step': [228, 456, 684, 912],
              'train_loss': [0.4255075752735138,
               0.5024822354316711,
               0.35160031914711,
               0.8196984529495239],
              'mcc': [np.float64(0.24554666774682865),
               np.float64(0.29825033774333287),
               np.float64(0.27810187614999654),
               np.float64(0.309803794155198)],
              'accuracy': [0.6938271604938272,
               0.6962962962962963,
               0.6987654320987654,
               0.7012345679012346],
              'f1_score': [0.5910956227201667,
               0.6465040696560485,
               0.6256364407807007,
               0.6522519709624541],
              'tp': [np.int64(242),
               np.int64(217),
               np.int64(231),
               np.int64(218)],
              'tn': [np.int64(39), np.int64(65), np.int64(52), np.int64(66)],
              'fp': [np.int64(99), np.int64(73), np.int64(86),

In [9]:
# Get ALBERT raw logits
albert_logits, _ = albert_model.predict(eval_df["text"].tolist())

from scipy.special import expit  # sigmoid

# Convert logits to probabilities for binary classification
probs_class1 = expit(albert_logits)  # sigmoid for class 1
probs_class0 = 1 - probs_class1      # remaining for class 0

# Combine into (N, 2) shape
albert_probs = np.vstack((probs_class0, probs_class1)).T



  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

In [10]:
# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(train_df["text"]).toarray()
y_train_keras = to_categorical(train_df["labels"])

# ANN Model
ann_model = Sequential()
ann_model.add(Dense(256, input_dim=5000, activation='relu'))
ann_model.add(Dense(num_labels, activation='softmax'))

ann_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
ann_model.fit(X_train_tfidf, y_train_keras, epochs=5, batch_size=32, verbose=1)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.6616 - loss: 0.6343
Epoch 2/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7835 - loss: 0.4525
Epoch 3/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8966 - loss: 0.2888
Epoch 4/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9470 - loss: 0.1769
Epoch 5/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9692 - loss: 0.1096


In [11]:
X_eval_tfidf = tfidf.transform(eval_df["text"]).toarray()
keras_probs = ann_model.predict(X_eval_tfidf)


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step


In [12]:
# Average the probabilities


ensemble_probs = (albert_probs + keras_probs) / 2
ensemble_preds = np.argmax(ensemble_probs, axis=1)
ensemble_labels = label_encoder.inverse_transform(ensemble_preds)


In [14]:
import pandas as pd
import json

# Create the submission DataFrame
submission_df = pd.DataFrame({
    "index": eval_df["index"],
    "prediction": ensemble_labels
})

# Convert DataFrame to list of dictionaries
submission_data = submission_df.to_dict(orient='records')

# Save to JSON
save_path = '/content/drive/MyDrive/SubTaskD.json'
with open(save_path, 'w') as json_file:
    json.dump(submission_data, json_file, indent=2)

print(f"✅ Submission file saved to: {save_path}")



✅ Submission file saved to: /content/drive/MyDrive/SubTaskD.json
